# Image cropper
Converts rectangular images to square

In [1]:
import cv2
import matplotlib.pyplot as plt
import json

In [2]:
def square_image(data, in_folder_path, out_folder_path, change_ext=None):
    
    n_images = len(data["annotations"])
    for image in data["annotations"]:
        img_id = image["image_id"] 
        img_name = data["images"][img_id]["file_name"] #Considering image ids are in order
        # Change the extension we are looking for if required
        if change_ext is not None:
            img_name = "{}.{}".format(img_name.split(".")[0], change_ext)
        # Get bounding box
        bx,by,bw,bh = image["bbox"] #(x,y,w,h)
        # Calculate the center of the bounding box
        center_x,center_y = int(bx+bw/2),int(by+bh/2)
        # Get img path
        img_path = "{}/{}".format(in_folder_path, img_name)
        # Read the image
        img = cv2.imread(img_path)
        # Get the iamge size
        h,w,c = img.shape
        # We are creating a square, and cropping the horizontal axis, we calculate w_min and w_max using bbox centers
        w_min = int(center_x - h/2)
        w_max = w_min + h
        # print(w_min,w_max)
        # If the predicted crop limits are out of image boundaries we recalculate them
        if w_min < 0:
            w_min = 0
            w_max = h
        elif w_max > w:
            w_max = w
            w_min = w-h
        # print(w_min,w_max)
        
        # Cropping the image
        crop_img = img[0:h, w_min:w_max]
        # Saving cropped images
        if change_ext is not None:
            dest_path = "{}/{}_crop.{}".format(out_folder_path, img_name.split(".")[0], change_ext)
        else:
            dest_path = "{}/{}_crop.{}".format(out_folder_path, img_name.split(".")[0], img_name.split(".")[-1])
        cv2.imwrite(dest_path,crop_img)
        # Printing the progress
        sys.stdout.write('\r'+  "Done: {}/{}".format(img_id +1,n_images))
        sys.stdout.flush()

In [3]:
ann_file = "/home/josmar/proyectos/codes/datasets/ucb_gait_frames/annotations/ucb_gait_poly.json"
with open(ann_file) as f:
    ann_data = json.load(f)


In [4]:
in_folder_path = "/home/josmar/proyectos/codes/datasets/ucb_gait_frames/silhouettes"
out_folder_path =  "/home/josmar/proyectos/codes/datasets/ucb_gait_square/silhouettes"
square_image(ann_data, in_folder_path, out_folder_path, change_ext="png")

Done: 12300/12300

### Checking if the sizes are correct

In [5]:
results_list = os.listdir(out_folder_path)
res_sizes = [cv2.imread(os.path.join(out_folder_path,i)).shape for i in results_list]
print(set(res_sizes))

{(720, 720, 3)}
